# EP

### But :  Réserver plusieurs noeuds d'un cluster et lancer smpirun ep à l'aide de "execo.action.Remote" 


#### Préparation de l'env.

In [1]:
%%bash -s
pip3 install --user execo


In [2]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import re
from shutil import copy
from subprocess import check_output

Réservation noeuds

In [3]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 5
# Temps réservation
walltime = "6:0:0" 

# filters out Nancy's graphene cluster (buggy)
resources_selection = "-p \"cluster != 'graphene'\"" 

# Nancy
site = "nancy"

In [5]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

A   job is already running, using it. jobid is 1565034


On les range

In [12]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('grisou-1.nancy.grid5000.fr'),
 Host('grisou-10.nancy.grid5000.fr'),
 Host('grisou-11.nancy.grid5000.fr'),
 Host('grisou-12.nancy.grid5000.fr'),
 Host('grisou-13.nancy.grid5000.fr')]

In [13]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = './debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [ ]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

In [16]:
# execo.action.Remote(cmd,hosts,connection_params,process_args)
Remote_test = execo.action.Remote(cmd='ps',hosts=nodes,connection_params=None, process_args=None)

In [17]:
Remote_test.run()

Remote('ps', [Host('grisou-1.nancy.grid5000.fr'), Host('grisou-10.nancy.grid5000.fr'), Host('grisou-11.nancy.grid5000.fr'), Host('grisou-12.nancy.grid5000.fr'), Host('grisou-13.nancy.grid5000.fr')])

In [ ]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j 4 && make check && make install -j 4',hosts=nodes,connection_params={'user':'root'})

In [ ]:
Remote_install.run().ok

In [ ]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=["griffon.xml","griffon_hostfile.txt","ep.c","nas_common.h","nas_common.c","Makefile"],connection_params={'user':'root'})
Remote_smpi_env.run().ok

In [ ]:
Remote_smpi = execo.action.Remote(cmd ='make && make run',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

In [22]:
Remote_retrieve = execo.action.Get(hosts=nodes,remote_files=["smpi_results.txt"], local_location='.',connection_params={'user':'root'})
Remote_retrieve.run().ok

True

In [23]:
execo.Process("cat smpi_results.txt").run().stdout

' NAS Parallel Benchmarks 3.2 -- EP Benchmark Number of random numbers generated: 536870912\n Number of active processes: 12\nEP Benchmark Results:\nCPU Time=1\nN = 2^28\nNo. Gaussain Pairs =510114\nSum = 154.017306 -2260\nCount:0\t 1\n1\t 463600\n2\t 86044\n3\t 5549\n4\t 84\n5\t 5\n6\t 14\n7\t 13\n8\t 6\n9\t 10\n\n\n EP Benchmark Completed\n Class           =                        A\n Size            =                       29\n Iterations      =                        0\n Time in seconds =                     1.76\n Total processes =                       12\n Compiled procs  =                        8\n Mop/s total     =                305770.64\n Mop/s/process   =                 25480.89\n Operation type  =  Random number generated\n Verification    =             UNSUCCESSFUL\nTotal time:     0.001756\nGaussian pairs: 0.000000\nRandom numbers: 0.000000\n'